# 图生图-ControlNet-Canny

同济子豪兄 2023-6-13

MMagic文档：https://github.com/open-mmlab/mmagic/tree/main/configs/controlnet

ControlNet官方Repo：https://github.com/lllyasviel/ControlNet

## 进入 MMagic 主目录

In [1]:
import os
os.chdir('mmagic')

## 导入工具包

In [2]:
import cv2
import numpy as np
import mmcv
from mmengine import Config
from PIL import Image

from mmagic.registry import MODELS
from mmagic.utils import register_all_modules

register_all_modules()

2023-06-14 09:15:25.932463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 09:15:26.798977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /environment/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:
2023-06-14 09:15:26.799064: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /environment/miniconda3/lib/python3.8/site-packages/cv2/../..

## 载入ControlNet模型

In [3]:
cfg = Config.fromfile('configs/controlnet/controlnet-canny.py')
controlnet = MODELS.build(cfg.model).cuda()

06/14 09:15:28 - mmengine - INFO - Set model dtype to 'torch.float32'.
06/14 09:15:31 - mmengine - INFO - Creating runwayml/stable-diffusion-v1-5 by 'HuggingFace'
Do not support Xformers. Please install Xformers first. The program will run without Xformers.
06/14 09:15:33 - mmengine - INFO - Set model dtype to 'torch.float32'.
Do not support Xformers. Please install Xformers first. The program will run without Xformers.


## 输入Canny边缘图

In [4]:
control_url = 'https://user-images.githubusercontent.com/28132635/230288866-99603172-04cb-47b3-8adb-d1aa532d1d2c.jpg'
control_img = mmcv.imread(control_url)
control = cv2.Canny(control_img, 100, 200)
control = control[:, :, None]
control = np.concatenate([control] * 3, axis=2)
control = Image.fromarray(control)

In [5]:
# control

## 咒语Prompt

In [6]:
prompt = 'Room with blue walls and a yellow ceiling.'

## 执行预测

In [7]:
output_dict = controlnet.infer(prompt, control=control)
samples = output_dict['samples']
for idx, sample in enumerate(samples):
    sample.save(f'sample_{idx}.png')
controls = output_dict['controls']
for idx, control in enumerate(controls):
    control.save(f'control_{idx}.png')

100%|██████████| 20/20 [00:01<00:00, 10.31it/s]
